<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/Langchain_Memory_Persistent_Store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 简介

该Python Notebook演示LangChain框架所提供的Memory持久化能力。LangChain通过langchain.memory.chat_message_histories包中提供的一系列组件支持多种形式的历史消息存储，包括文件，数据库等。

本示例演示文件存储形式的持久化。

In [23]:
!pip install langchain openai --quiet --upgrade

In [24]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.callbacks import get_openai_callback
from langchain.memory import ConversationBufferMemory

1. 准备**OpenAI API Key**，**track_tokens_usage**辅助函数（调用`chain.run`函数，并统计token开销），以及LLM实例

In [26]:
import os

os.environ['OPENAI_API_KEY'] = 'your openai api key'

In [27]:
def track_tokens_usage(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Total tokens: {cb.total_tokens}')

    return result


In [28]:
llm = OpenAI(
  temperature=0,
	openai_api_key=os.environ["OPENAI_API_KEY"],
	model_name="text-davinci-003"
)


2. 使用Memory组件**ConversationBufferMemory**的默认配置

In [29]:
conversation = ConversationChain(llm=llm, memory = ConversationBufferMemory())

In [30]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [31]:
track_tokens_usage(conversation, "What is Langchain?")


Total tokens: 110


' Langchain is a blockchain-based language learning platform that allows users to learn new languages in a secure and decentralized environment. It uses a combination of blockchain technology and artificial intelligence to provide users with an immersive language learning experience.'

3. 引入消息历史持久化组件

In [32]:
from langchain.memory.chat_message_histories import FileChatMessageHistory

In [33]:
message_history = FileChatMessageHistory(file_path = 'conversation_20230620.txt')

memory = ConversationBufferMemory(chat_memory=message_history)

In [34]:
conversation = ConversationChain(llm=llm, memory = memory)

In [35]:
track_tokens_usage(conversation, "How does ChatGPT enable coherent conversation?")

Total tokens: 133


' ChatGPT is a natural language processing (NLP) model that enables coherent conversation by understanding the context of a conversation and providing relevant responses. It uses a transformer-based architecture to generate responses that are based on the context of the conversation. It also uses a large corpus of data to generate more accurate responses.'

In [36]:
!cat conversation_20230620.txt

[{"type": "human", "data": {"content": "How does ChatGPT enable coherent conversation?", "additional_kwargs": {}, "example": false}}, {"type": "ai", "data": {"content": " ChatGPT is a natural language processing (NLP) model that enables coherent conversation by understanding the context of a conversation and providing relevant responses. It uses a transformer-based architecture to generate responses that are based on the context of the conversation. It also uses a large corpus of data to generate more accurate responses.", "additional_kwargs": {}, "example": false}}]

In [37]:
track_tokens_usage(conversation, "Bye now!")

Total tokens: 156


' Bye! It was nice talking to you. Have a great day!'

In [38]:
!cat conversation_20230620.txt

[{"type": "human", "data": {"content": "How does ChatGPT enable coherent conversation?", "additional_kwargs": {}, "example": false}}, {"type": "ai", "data": {"content": " ChatGPT is a natural language processing (NLP) model that enables coherent conversation by understanding the context of a conversation and providing relevant responses. It uses a transformer-based architecture to generate responses that are based on the context of the conversation. It also uses a large corpus of data to generate more accurate responses.", "additional_kwargs": {}, "example": false}}, {"type": "human", "data": {"content": "Bye now!", "additional_kwargs": {}, "example": false}}, {"type": "ai", "data": {"content": " Bye! It was nice talking to you. Have a great day!", "additional_kwargs": {}, "example": false}}]